# 쇼핑몰 리뷰 감성 분석: Full Fine-Tuning vs PEFT (LoRA)

## 1. 프로젝트 개요
- **목표**: 패션 도메인 리뷰의 감성(긍정/중립/부정)을 분석하는 모델 최적화.
- **Base Model**: `klue/roberta-base`
- **비교 대상**: Full Fine-Tuning (전체 학습) vs LoRA (효율적 학습)
- **환경**: Local (M1 Debugging) / Colab (Full Training)

In [5]:
# =========================================================
# 환경 설정 및 프로젝트 폴더 연결
# =========================================================
import sys
import os
from pathlib import Path
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# 1. 환경 감지 및 설정
if 'google.colab' in sys.modules:
    logger.info("☁️ Google Colab 환경 감지됨.")

    # 필수 라이브러리 설치 (필요시 주석 해제)
    # !pip install -q transformers peft datasets evaluate accelerate

    from google.colab import drive
    drive.mount('/content/drive')

    # 프로젝트 폴더 탐색 (하드코딩 방지)
    TARGET_DIR_NAME = "13_SentimentAnalysis_Project"
    SEARCH_ROOT = Path("/content/drive/MyDrive")

    found_path = None
    try:
        # 깊이 2까지만 빠르게 검색
        patterns = [f"{TARGET_DIR_NAME}", f"*/{TARGET_DIR_NAME}", f"*/*/{TARGET_DIR_NAME}"]
        for pattern in patterns:
            results = list(SEARCH_ROOT.glob(pattern))
            if results:
                found_path = results[0]
                break
    except Exception as e:
        logger.error(f"검색 중 오류: {e}")

    if found_path and found_path.exists():
        os.chdir(found_path)
        logger.info(f"📂 작업 디렉토리 변경 완료: {os.getcwd()}")
    else:
        raise FileNotFoundError(f"'{TARGET_DIR_NAME}' 폴더를 찾을 수 없습니다.")

else:
    logger.info("💻 로컬 환경 감지됨. 현재 위치를 유지합니다.")

# 2. 경로 및 디바이스 설정
import torch

def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

device = get_device()
logger.info(f"🚀 사용 디바이스: {device}")

# 로컬 Mac M1 등에서 테스트 시 데이터 로드 제한 설정 (디버깅용)
is_mac_local = (device.type == 'mps')
SAMPLE_LIMIT = 2000 if is_mac_local else None

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# =========================================================
# 데이터 로드 및 전처리 함수 정의 (최종 강화판)
# =========================================================
import json
import pandas as pd
import unicodedata
import sys
import os
import logging
from pathlib import Path
from sklearn.model_selection import GroupShuffleSplit
from datasets import Dataset, DatasetDict

# 로거 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------------------------------------
# [Helper] 문자열 정규화 함수 (자소 분리 방지)
# ---------------------------------------------------------
def nfc(s: str) -> str:
    """문자열을 NFC(완성형) 표준으로 변환합니다."""
    return unicodedata.normalize("NFC", str(s))

# ---------------------------------------------------------
# 1. 데이터 루트 경로 찾기 ('쇼핑몰' 폴더 찾기)
# ---------------------------------------------------------
def robust_find_data_path():
    # CASE A: Google Colab (하드코딩)
    if 'google.colab' in sys.modules:
        colab_path = Path("/content/drive/MyDrive/codeit/13_SentimentAnalysis_Project/data/쇼핑몰")
        if colab_path.exists():
            logger.info(f"📍 Colab 환경: {colab_path}")
            return colab_path
        else:
            # 혹시 상위 폴더 이름이 깨졌을 수도 있으니, 상위에서 탐색 시도
            logger.warning("⚠️ 지정된 경로를 찾지 못해 상위 폴더에서 탐색을 시도합니다.")
            base_dir = Path("/content/drive/MyDrive/codeit/13_SentimentAnalysis_Project/data")

    # CASE B: 로컬/기타 (상대 경로 탐색)
    else:
        base_dir = Path(os.getcwd()) / "data"
        if not base_dir.exists():
            base_dir = Path(os.getcwd()).parent / "data"

    # 공통: base_dir 안에서 '쇼핑몰' 찾기 (NFC 정규화 적용)
    if not base_dir.exists():
        raise FileNotFoundError(f"❌ 'data' 폴더를 찾을 수 없습니다. ({base_dir})")

    target_keyword = "쇼핑몰"
    for child in base_dir.iterdir():
        if child.is_dir():
            if target_keyword in nfc(child.name): # 여기가 핵심! 폴더명을 NFC로 바꿔서 비교
                logger.info(f"✅ 타겟 데이터 폴더 발견: {child}")
                return child

    raise FileNotFoundError("❌ 'data' 폴더 내에서 '쇼핑몰' 폴더를 찾지 못했습니다.")

# ---------------------------------------------------------
# 2. JSON 파일 리스트 로드 및 필터링 (보여주신 코드 로직 반영)
# ---------------------------------------------------------
def load_json_files(data_dir, include_keywords=["패션"]):
    """
    data_dir 내의 모든 json을 찾은 뒤,
    경로(파일명 포함)에 include_keywords(예: '패션')가 포함된 것만 리턴합니다.
    """
    data_dir = Path(data_dir)
    all_files = list(data_dir.rglob("*.json"))

    if not all_files:
        logger.warning("📂 폴더 내에 JSON 파일이 하나도 없습니다.")
        return []

    # [핵심 로직] 사용자분이 작성한 코드 반영: 경로 전체를 NFC로 변환 후 키워드 검사
    filtered_files = [
        p for p in all_files
        if any(k in nfc(str(p)) for k in include_keywords)
    ]

    logger.info(f"📦 전체 JSON: {len(all_files)}개 -> 필터링 후({include_keywords}): {len(filtered_files)}개")

    # 만약 필터링 결과가 0개라면, 키워드 문제일 수 있으니 경고
    if len(all_files) > 0 and len(filtered_files) == 0:
        logger.warning(f"⚠️ 주의: 파일은 존재하지만 '{include_keywords}' 키워드 조건에 맞는 파일이 없습니다.")

    return filtered_files

# ---------------------------------------------------------
# 3. 데이터 파싱 (DataFrame 변환)
# ---------------------------------------------------------
def parse_data(file_paths, sample_limit=None):
    data = []
    if sample_limit:
        file_paths = file_paths[:sample_limit]
        logger.info(f"⚡ 샘플링 모드: {len(file_paths)}개만 로드합니다.")

    for file_path in file_paths:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = json.load(f)

                # JSON이 리스트 형태이므로 그대로 순회합니다.
                if isinstance(content, list):
                    items = content
                else:
                    items = [content]

                for item in items:
                    # [핵심] 확인된 실제 키 값으로 매핑
                    # RawText: 리뷰 원문
                    # GeneralPolarity: 전체 긍부정 (1: 긍정, 0: 중립, -1: 부정 등)
                    text = item.get('RawText')
                    label = item.get('GeneralPolarity')

                    # 그룹 ID: 상품명(ProductName)이 있으면 더 정확하므로 이를 우선 사용
                    group_id = item.get('ProductName') or nfc(file_path.parent.name)

                    if text is not None and label is not None:
                        data.append({
                            'text': text,
                            'label': label,
                            'group_id': group_id
                        })

        except Exception as e:
            logger.debug(f"파일 로드 실패 ({file_path.name}): {e}")
            continue

    df = pd.DataFrame(data)

    # 데이터가 여전히 비어있는지 체크
    if df.empty:
        logger.error("🚨 데이터를 파싱했지만 결과가 비어있습니다. 키 값을 다시 확인하세요.")
    else:
        logger.info(f"📊 데이터프레임 생성 완료: {len(df)} 행")

    return df

# ---------------------------------------------------------
# 4. 데이터 누수 방지 분할 (Group Split)
# ---------------------------------------------------------
def split_data_leakage_proof(df, test_size=0.2):
    if df.empty:
        raise ValueError("데이터프레임이 비어있습니다.")

    splitter = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
    train_idx, test_idx = next(splitter.split(df, groups=df['group_id']))

    train_df = df.iloc[train_idx].copy()
    test_df = df.iloc[test_idx].copy()

    logger.info(f"✂️ 데이터 분할 완료 - Train: {len(train_df)}, Test: {len(test_df)}")
    return train_df, test_df

# ---------------------------------------------------------
# 5. HuggingFace Dataset 변환
# ---------------------------------------------------------
def create_dataset_dict(train_df, test_df):
    train_ds = Dataset.from_pandas(train_df[['text', 'label']])
    test_ds = Dataset.from_pandas(test_df[['text', 'label']])
    return DatasetDict({'train': train_ds, 'test': test_ds})

## 2. 유틸리티 및 데이터 로더 (Self-Contained)

In [10]:
# =========================================================
# [Cell 3] 실행 파이프라인
# =========================================================
try:
    # 1. 경로 탐색
    DATA_DIR = robust_find_data_path()

    # 2. 파일 로드
    json_files = load_json_files(DATA_DIR)

    if not json_files:
        raise ValueError("처리할 파일이 없습니다.")

    # 3. 파싱 (로컬 환경이면 자동 샘플링 적용됨)
    df = parse_data(json_files, sample_limit=SAMPLE_LIMIT)

    if df.empty:
        raise ValueError("데이터프레임이 비어있습니다. JSON 키 값(text/label)을 확인해주세요.")

    # 4. 분할
    train_df, test_df = split_data_leakage_proof(df)

    # 5. 데이터셋 생성
    dataset = create_dataset_dict(train_df, test_df)

    print("\n✨ 최종 데이터셋 준비 완료!")
    print(dataset)
    print("\n[Train 샘플 확인]")
    print(dataset['train'][0])

except Exception as e:
    logger.error(f"❌ 실행 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()


✨ 최종 데이터셋 준비 완료!
DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 36529
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 7869
    })
})

[Train 샘플 확인]
{'text': '배송도빠르고  상품도 가격대비 가성비 굿~~ 너무 착용하기 편하고 가죽도 부드럽고 너무 좋습니다. 굿굿굿~~너무 맘에 듭니다^^', 'label': '1', '__index_level_0__': 0}


## 3. 전처리 (Preprocessing)

In [12]:
!pip install transformers datasets evaluate accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [14]:
import numpy as np
import evaluate
from transformers import AutoTokenizer, DataCollatorWithPadding
# ---------------------------------------------------------
# 5. 토크나이징 및 전처리
# ---------------------------------------------------------
MODEL_ID = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128, padding=False)

# 여기서 dataset 변수가 정상적으로 사용됩니다
encoded_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Preprocessing Complete.")

# Metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average="macro")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Map:   0%|          | 0/36529 [00:00<?, ? examples/s]

Map:   0%|          | 0/7869 [00:00<?, ? examples/s]

Preprocessing Complete.


## 4. Full Fine-Tuning

In [ ]:
model_ft = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=3)

training_args = TrainingArguments(
    output_dir="./results/full_ft",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3 if not is_mac_local else 1,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(), # Use Mixed Precision on CUDA
    report_to="none",
    gradient_accumulation_steps=2,
)

trainer_ft = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

if is_mac_local:
    print("!!! Local Debugging Mode: Skipping Full Training !!!")
    # trainer_ft.train() # Uncomment to debug run
else:
    trainer_ft.train()
    trainer_ft.save_model("./saved_models/full_ft")

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-547381384.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_ft = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.671717,0.742291,0.614400
2,No log,0.386439,0.856828,0.822034
3,No log,0.395527,0.852423,0.805895


## 5. PEFT (LoRA)

In [ ]:
# Reset to base model for Fairness
model_lora = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=3)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8, # Rank
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "value"] # Target Attention Layers
)

model_lora = get_peft_model(model_lora, lora_config)
model_lora.print_trainable_parameters()

training_args_lora = TrainingArguments(
    output_dir="./results/lora",
    learning_rate=1e-4, # Higher LR for LoRA
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3 if not is_mac_local else 1,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

if is_mac_local:
    print("!!! Local Debugging Mode: Skipping LoRA Training !!!")
else:
    trainer_lora.train()
    trainer_lora.save_model("./saved_models/lora_ft")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-641167954.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(


trainable params: 887,811 || all params: 111,508,230 || trainable%: 0.7962


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.751021,0.588106,0.246879
2,No log,0.457338,0.797357,0.722738
3,No log,0.438757,0.821586,0.756631


## 6. 비교 분석 및 결과 Summary

> 인용구 추가


- Full FT와 LoRA의 F1 Score, Training Time, Model Size를 비교합니다.

In [ ]:
import os
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import torch

# 결과 저장 폴더 생성
RESULTS_DIR = "./results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# ---------------------------------------------------------
# 1. 로그 데이터 수집 (사용자님의 로그 기반 하드코딩)
# ---------------------------------------------------------
# Full FT: 1m 20s = 80s, Best F1: 0.8220 (Epoch 2)
# LoRA: 33s, Best F1: 0.7566 (Epoch 3)

comparison_data = {
    "Model": ["Full Fine-Tuning", "LoRA"],
    "Best F1 Score": [0.8220, 0.7566],
    "Best Accuracy": [0.8568, 0.8215],
    "Training Time (sec)": [80, 33],        # 1분 20초 vs 33초
    "Trainable Params": [111508230, 887811] # 약 1.1억개 vs 88만개
}

df_results = pd.DataFrame(comparison_data)

# 학습 곡선 데이터 (Epoch별 F1 변화)
history_data = {
    "Epoch": [1, 2, 3],
    "Full FT F1": [0.6144, 0.8220, 0.8058],
    "LoRA F1":    [0.2468, 0.7227, 0.7566]
}
df_history = pd.DataFrame(history_data)

# ---------------------------------------------------------
# 2. 정량적 분석: Plotly 시각화
# ---------------------------------------------------------
print("📊 결과 시각화를 생성합니다...")

# (1) 성능(F1) vs 속도(Time) 비교 차트
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

# Bar: F1 Score
fig1.add_trace(
    go.Bar(x=df_results["Model"], y=df_results["Best F1 Score"], name="Best F1 Score", marker_color=['#636EFA', '#EF553B']),
    secondary_y=False
)

# Line: Training Time
fig1.add_trace(
    go.Scatter(x=df_results["Model"], y=df_results["Training Time (sec)"], name="Training Time (s)", mode='lines+markers', line=dict(color='green', dash='dot')),
    secondary_y=True
)

fig1.update_layout(title_text="<b>[Performance vs Efficiency]</b> Full FT vs LoRA")
fig1.update_yaxes(title_text="F1 Score", secondary_y=False, range=[0, 1.0])
fig1.update_yaxes(title_text="Training Time (sec)", secondary_y=True)

# (2) 학습 곡선 비교 (Epochs vs F1)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_history["Epoch"], y=df_history["Full FT F1"], mode='lines+markers', name='Full FT'))
fig2.add_trace(go.Scatter(x=df_history["Epoch"], y=df_history["LoRA F1"], mode='lines+markers', name='LoRA'))
fig2.update_layout(title="<b>[Learning Curve]</b> Epoch vs F1 Score", xaxis_title="Epoch", yaxis_title="F1 Score")

# (3) 파라미터 효율성 (Log Scale Bar)
fig3 = px.bar(df_results, x="Model", y="Trainable Params", title="<b>[Parameter Efficiency]</b> Trainable Parameters (Log Scale)", log_y=True, text="Trainable Params", color="Model")
fig3.update_traces(texttemplate='%{text:.2s}', textposition='outside')

# 차트 출력
fig1.show()
fig2.show()
fig3.show()

# 차트 저장 (HTML)
fig1.write_html(f"{RESULTS_DIR}/comparison_performance.html")
fig2.write_html(f"{RESULTS_DIR}/comparison_learning_curve.html")
print(f"✅ 차트 저장 완료: {RESULTS_DIR}/")

# ---------------------------------------------------------
# 3. 정성적 분석: 실제 추론 결과 비교 (Inference)
# ---------------------------------------------------------
print("\n🔍 [정성적 분석] 실제 문장 추론 테스트")

test_sentence = "배송도 정말 빠르고 디자인도 사진이랑 똑같아요! 다만 마감이 살짝 아쉽네요."
# 예상: 긍정(2) 혹은 중립(1). '마감이 아쉽다'는 부정 요소가 섞여 있어 모델의 판단이 중요함.

def predict_sentiment(model, text):
    model.eval()
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    top_prob, top_label = torch.max(probs, dim=-1)
    return top_label.item(), top_prob.item()

# Label Map
id2label = {0: "부정(Negative)", 1: "중립(Neutral)", 2: "긍정(Positive)"}

# Full FT 예측
ft_label, ft_conf = predict_sentiment(trainer_ft.model, test_sentence)
# LoRA 예측
lora_label, lora_conf = predict_sentiment(trainer_lora.model, test_sentence)

print(f"\n📝 테스트 문장: '{test_sentence}'")
print("-" * 50)
print(f"🤖 Full FT 모델 예측 : {id2label[ft_label]} (확신도: {ft_conf*100:.2f}%)")
print(f"⚡ LoRA 모델 예측    : {id2label[lora_label]} (확신도: {lora_conf*100:.2f}%)")
print("-" * 50)

📊 결과 시각화를 생성합니다...


✅ 차트 저장 완료: ./results/

🔍 [정성적 분석] 실제 문장 추론 테스트

📝 테스트 문장: '배송도 정말 빠르고 디자인도 사진이랑 똑같아요! 다만 마감이 살짝 아쉽네요.'
--------------------------------------------------
🤖 Full FT 모델 예측 : 중립(Neutral) (확신도: 67.14%)
⚡ LoRA 모델 예측    : 긍정(Positive) (확신도: 74.81%)
--------------------------------------------------


In [ ]:
import pandas as pd

def extract_best_metrics(trainer, name):
    # 로그 히스토리에서 'eval_f1'이 있는 기록만 가져오기
    eval_logs = [log for log in trainer.state.log_history if 'eval_f1' in log]

    # F1 점수가 가장 높은 기록 찾기
    if eval_logs:
        best_log = max(eval_logs, key=lambda x: x['eval_f1'])
        best_f1 = best_log['eval_f1']
        best_acc = best_log['eval_accuracy']
        best_epoch = best_log['epoch']
    else:
        best_f1, best_acc, best_epoch = 0, 0, 0

    # 총 학습 시간 (로그의 마지막 부분에 보통 기록됨)
    # 없을 경우를 대비해 안전하게 처리
    try:
        train_runtime = trainer.state.log_history[-1].get('train_runtime', 0)
    except:
        train_runtime = 0

    # 파라미터 수 계산
    total_params = sum(p.numel() for p in trainer.model.parameters())
    trainable_params = sum(p.numel() for p in trainer.model.parameters() if p.requires_grad)

    return {
        "Model": name,
        "Best F1": best_f1,
        "Best Accuracy": best_acc,
        "Best Epoch": best_epoch,
        "Training Time(s)": round(train_runtime, 2),
        "Trainable Params": f"{trainable_params:,} ({(trainable_params/total_params)*100:.1f}%)"
    }

# 1. 실제 데이터 추출
res_ft = extract_best_metrics(trainer_ft, "Full Fine-Tuning")
res_lora = extract_best_metrics(trainer_lora, "LoRA")

# 2. DataFrame으로 보기 좋게 출력
df_real_result = pd.DataFrame([res_ft, res_lora])
display(df_real_result)

,Model,Best F1,Best Accuracy,Best Epoch,Training Time(s),Trainable Params
0,Full Fine-Tuning,0.822034,0.856828,2.0,84.38,"110,620,419 (100.0%)"
1,LoRA,0.756631,0.821586,3.0,34.15,"887,811 (0.8%)"


Learning Curve (Fig 2) 확인:

Full FT는 1 Epoch부터 이미 성능이 높게 나옵니다.

LoRA는 1 Epoch에서 성능이 매우 낮았다가(0.24) 2 Epoch부터 급격히 따라잡는 모습을 보입니다. 이는 LoRA가 초기 가중치 정렬에 시간이 조금 더 필요함을 의미합니다. Epoch를 5~10회로 늘리면 Full FT와 거의 비슷해질 가능성이 높습니다.

정성적 분석 결과:

test_sentence("배송 빠르고... 마감이 아쉽다")는 복합적인 감정이 섞여 있습니다. 두 모델이 미세하게 다른 확신도(Confidence)를 보이는지 확인해보세요.

In [ ]:
import torch
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch.nn.functional as F

# 1. 다양한 테스트 문장 리스트 (난이도별 배치)
test_sentences = [
    "배송도 빠르고 디자인도 너무 마음에 들어요! 강추합니다.",            # [명확한 긍정]
    "최악이에요. 다신 안 삽니다. 환불해주세요.",                       # [명확한 부정]
    "그냥 평범해요. 가격 대비 쏘쏘한 느낌?",                           # [중립/애매]
    "디자인은 정말 예쁜데, 마감이 너무 엉망이라 실망했어요.",             # [복합: 긍정+부정 -> 보통 부정이나 중립]
    "배송이 늦어서 화날 뻔했는데, 막상 입어보니 예뻐서 용서가 되네요.",    # [복합: 부정->긍정]
    "도대체 언제 도착하나요? 일주일째 기다리고 있습니다."                  # [문의/불만 -> 부정]
]

labels_map = {0: "부정(Neg)", 1: "중립(Neu)", 2: "긍정(Pos)"}

# 2. 확률 추출 함수 (Softmax 적용)
def get_probabilities(model, text, tokenizer, device):
    model.eval()
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Logits -> Probabilities (0~1 사이 값으로 변환)
    probs = F.softmax(outputs.logits, dim=-1)[0].cpu().numpy()
    return probs # [prob_neg, prob_neu, prob_pos]

# 3. Full FT vs LoRA 추론 실행
results = []

print("🔎 상세 추론 분석을 진행합니다...")

for text in test_sentences:
    # Full FT 예측
    ft_probs = get_probabilities(trainer_ft.model, text, tokenizer, device)
    # LoRA 예측
    lora_probs = get_probabilities(trainer_lora.model, text, tokenizer, device)

    results.append({
        "문장": text,
        # Full FT 결과
        "FT_Neg": ft_probs[0], "FT_Neu": ft_probs[1], "FT_Pos": ft_probs[2],
        "FT_Pred": labels_map[ft_probs.argmax()],
        # LoRA 결과
        "LoRA_Neg": lora_probs[0], "LoRA_Neu": lora_probs[1], "LoRA_Pos": lora_probs[2],
        "LoRA_Pred": labels_map[lora_probs.argmax()]
    })

df_analysis = pd.DataFrame(results)

# 4. 결과 테이블 출력 (가독성 좋게 퍼센트로 변환)
pd.options.display.float_format = '{:.1%}'.format
print("\n📊 [모델별 클래스 확률 분포 비교]")
display(df_analysis[["문장", "FT_Pred", "FT_Neg", "FT_Neu", "FT_Pos", "LoRA_Pred", "LoRA_Neg", "LoRA_Neu", "LoRA_Pos"]])

# 5. 시각화 (Plotly Grouped Bar Chart)
print("\n📈 시각화 생성 중...")

# 복잡한 그래프를 위해 데이터 구조 변경
fig = make_subplots(
    rows=len(test_sentences), cols=1,
    subplot_titles=[f"Q{i+1}: {t[:20]}..." for i, t in enumerate(test_sentences)],
    vertical_spacing=0.05
)

colors = {'부정': '#EF553B', '중립': '#FECB52', '긍정': '#00CC96'} # Red, Yellow, Green

for i, row in df_analysis.iterrows():
    # Full FT Bars
    fig.add_trace(go.Bar(
        name='Full FT',
        x=['부정', '중립', '긍정'],
        y=[row['FT_Neg'], row['FT_Neu'], row['FT_Pos']],
        marker_color=[colors['부정'], colors['중립'], colors['긍정']],
        opacity=0.9,
        text=[f"{v:.1%}" for v in [row['FT_Neg'], row['FT_Neu'], row['FT_Pos']]],
        textposition='auto',
        legendgroup='Full FT',
        showlegend=(i==0) # 범례는 처음에만 표시
    ), row=i+1, col=1)

    # LoRA Bars (비교를 위해 옆에 얇게 혹은 겹쳐서 표현하고 싶으나, 여기선 별도 Trace로)
    # 비교를 명확히 하기 위해 LoRA는 투명도 조절 or 패턴 사용
    fig.add_trace(go.Bar(
        name='LoRA',
        x=['부정', '중립', '긍정'],
        y=[row['LoRA_Neg'], row['LoRA_Neu'], row['LoRA_Pos']],
        marker_color=[colors['부정'], colors['중립'], colors['긍정']],
        opacity=0.4, # LoRA는 흐리게 표시해서 겹쳐 보이게 함
        text=[f"{v:.1%}" for v in [row['LoRA_Neg'], row['LoRA_Neu'], row['LoRA_Pos']]],
        textposition='outside',
        legendgroup='LoRA',
        showlegend=(i==0)
    ), row=i+1, col=1)

fig.update_layout(
    title_text="<b>[심층 분석] 문장별 감정 확률 분포 (진한색: Full FT / 연한색: LoRA)</b>",
    height=300 * len(test_sentences),
    barmode='group' # 그룹으로 묶어서 비교
)

fig.show()

🔎 상세 추론 분석을 진행합니다...

📊 [모델별 클래스 확률 분포 비교]


,문장,FT_Pred,FT_Neg,FT_Neu,FT_Pos,LoRA_Pred,LoRA_Neg,LoRA_Neu,LoRA_Pos
0,배송도 빠르고 디자인도 너무 마음에 들어요! 강추합니다.,긍정(Pos),0.7%,1.7%,97.6%,긍정(Pos),0.1%,0.9%,99.0%
1,최악이에요. 다신 안 삽니다. 환불해주세요.,부정(Neg),86.8%,5.8%,7.4%,부정(Neg),85.0%,6.5%,8.5%
2,그냥 평범해요. 가격 대비 쏘쏘한 느낌?,중립(Neu),27.2%,63.5%,9.2%,중립(Neu),26.7%,58.4%,14.9%
3,"디자인은 정말 예쁜데, 마감이 너무 엉망이라 실망했어요.",부정(Neg),64.1%,31.6%,4.3%,부정(Neg),70.6%,22.4%,7.0%
4,"배송이 늦어서 화날 뻔했는데, 막상 입어보니 예뻐서 용서가 되네요.",긍정(Pos),1.0%,11.0%,88.0%,긍정(Pos),6.6%,27.6%,65.8%
5,도대체 언제 도착하나요? 일주일째 기다리고 있습니다.,부정(Neg),69.9%,24.0%,6.0%,부정(Neg),89.8%,7.3%,2.9%



📈 시각화 생성 중...


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix

# 1. 전체 테스트 데이터셋에 대해 추론(Prediction) 실행
print("🚀 전체 테스트 데이터셋에 대해 추론을 시작합니다...")

# Full FT 예측
pred_output_ft = trainer_ft.predict(encoded_dataset["test"])
pred_labels_ft = np.argmax(pred_output_ft.predictions, axis=1)

# LoRA 예측
pred_output_lora = trainer_lora.predict(encoded_dataset["test"])
pred_labels_lora = np.argmax(pred_output_lora.predictions, axis=1)

# 실제 정답 (Ground Truth)
true_labels = np.array(encoded_dataset["test"]["label"])

# 라벨 매핑
label_map = {0: '부정', 1: '중립', 2: '긍정'}
labels_list = ['부정', '중립', '긍정']

# ---------------------------------------------------------
# 2. 분포 비교 (Distribution Comparison)
# ---------------------------------------------------------
# 각 리스트를 데이터프레임으로 변환하여 카운팅
df_dist = pd.DataFrame({
    'Actual': [label_map[l] for l in true_labels],
    'Full FT': [label_map[l] for l in pred_labels_ft],
    'LoRA': [label_map[l] for l in pred_labels_lora]
})

# 비율 계산 함수
def get_distribution(series, name):
    counts = series.value_counts(normalize=True).reindex(labels_list, fill_value=0) * 100
    df = counts.reset_index()
    df.columns = ['Sentiment', 'Percentage']
    df['Model'] = name
    return df

dist_actual = get_distribution(df_dist['Actual'], '실제 정답(Ground Truth)')
dist_ft = get_distribution(df_dist['Full FT'], 'Full FT 예측')
dist_lora = get_distribution(df_dist['LoRA'], 'LoRA 예측')

# 하나로 합치기
df_final_dist = pd.concat([dist_actual, dist_ft, dist_lora])

# 시각화 1: 분포 비교 막대 그래프
fig_dist = px.bar(
    df_final_dist,
    x="Sentiment",
    y="Percentage",
    color="Model",
    barmode="group",
    text_auto='.1f',
    title="<b>[전체 분포 비교] 실제 정답 vs 모델별 예측 비율</b>",
    color_discrete_map={'실제 정답(Ground Truth)': 'gray', 'Full FT 예측': '#636EFA', 'LoRA 예측': '#EF553B'}
)
fig_dist.update_layout(yaxis_title="비율 (%)")
fig_dist.show()

# ---------------------------------------------------------
# 3. 오답 분석 (Confusion Matrix)
# ---------------------------------------------------------
# 혼동 행렬 계산
cm_ft = confusion_matrix(true_labels, pred_labels_ft)
cm_lora = confusion_matrix(true_labels, pred_labels_lora)

# 시각화 2: Confusion Matrix Heatmap
fig_cm = make_subplots(
    rows=1, cols=2,
    subplot_titles=("<b>Full FT 혼동 행렬</b>", "<b>LoRA 혼동 행렬</b>"),
    horizontal_spacing=0.15
)

# Full FT Heatmap
fig_cm.add_trace(
    go.Heatmap(
        z=cm_ft, x=labels_list, y=labels_list,
        colorscale='Blues', text=cm_ft, texttemplate="%{text}", textfont={"size":16},
        name="Full FT"
    ), row=1, col=1
)

# LoRA Heatmap
fig_cm.add_trace(
    go.Heatmap(
        z=cm_lora, x=labels_list, y=labels_list,
        colorscale='Reds', text=cm_lora, texttemplate="%{text}", textfont={"size":16},
        name="LoRA"
    ), row=1, col=2
)

fig_cm.update_layout(
    title_text="<b>[오답 상세 분석] Confusion Matrix (X: 예측, Y: 실제)</b>",
    xaxis_title="모델의 예측 (Predicted)", yaxis_title="실제 정답 (Actual)",
    xaxis2_title="모델의 예측 (Predicted)", yaxis2_title="실제 정답 (Actual)"
)
fig_cm.show()

# ---------------------------------------------------------
# 4. 요약 텍스트 출력
# ---------------------------------------------------------
print("\n📝 [분석 코멘트 생성]")
print("-" * 50)
print(f"총 테스트 데이터 수: {len(true_labels)}개")

# 정확도 재확인
acc_ft = np.mean(pred_labels_ft == true_labels)
acc_lora = np.mean(pred_labels_lora == true_labels)

print(f"Full FT 정확도: {acc_ft:.2%}")
print(f"LoRA    정확도: {acc_lora:.2%}")

# 편향성 간단 체크
cnt_actual = df_dist['Actual'].value_counts()
cnt_ft = df_dist['Full FT'].value_counts()
cnt_lora = df_dist['LoRA'].value_counts()

print("\n[감정별 개수 비교 (실제 vs FT vs LoRA)]")
for label in labels_list:
    a = cnt_actual.get(label, 0)
    f = cnt_ft.get(label, 0)
    l = cnt_lora.get(label, 0)
    print(f"- {label}: 실제 {a}개 | FT {f}개 ({f-a:+d}) | LoRA {l}개 ({l-a:+d})")
print("-" * 50)

🚀 전체 테스트 데이터셋에 대해 추론을 시작합니다...



📝 [분석 코멘트 생성]
--------------------------------------------------
총 테스트 데이터 수: 454개
Full FT 정확도: 85.68%
LoRA    정확도: 82.16%

[감정별 개수 비교 (실제 vs FT vs LoRA)]
- 부정: 실제 88개 | FT 81개 (-7) | LoRA 103개 (+15)
- 중립: 실제 99개 | FT 106개 (+7) | LoRA 70개 (-29)
- 긍정: 실제 267개 | FT 267개 (+0) | LoRA 281개 (+14)
--------------------------------------------------


# 📊 [최종 리포트] 감정 분석 모델 성능 비교: Full Fine-Tuning vs. LoRA

## 1. 프로젝트 개요
- **목적**: 한국어 쇼핑몰 리뷰 감정 분석 (3-Class: 부정, 중립, 긍정)
- **비교 대상**:
  1. **Full Fine-Tuning (FT)**: 모델 전체 파라미터 업데이트 (전통적 방식)
  2. **LoRA (Low-Rank Adaptation)**: 핵심 가중치만 학습 (경량화 방식)
- **데이터셋**: Train 1,638건 / Test 454건

---

## 2. 정량적 성과 요약 (Quantitative Summary)

| 비교 항목 | Full Fine-Tuning | LoRA | 비고 |
|:---:|:---:|:---:|:---|
| **Best Accuracy** | **85.68%** | 82.16% | FT가 약 **3.5%p** 우세 |
| **Best F1 Score** | **0.8220** | 0.7566 | FT가 균형 잡힌 성능 보임 |
| **학습 시간** | 84.38초 | **34.15초** | LoRA가 약 **2.5배 빠름** |
| **학습 파라미터** | 1.1억 개 (100%) | **88만 개 (0.8%)** | LoRA가 메모리 효율 압도적 |
| **Best Epoch** | 2 Epoch | 3 Epoch | LoRA는 학습 수렴이 다소 느림 |

> **요약**: 정확도는 Full FT가 더 높지만, LoRA는 단 **0.8%의 파라미터**만으로 FT 성능의 **96% 수준**까지 도달하는 효율성을 증명함.

---

## 3. 예측 분포 및 편향성 분석 (Distribution Analysis)
테스트 데이터 454개에 대한 두 모델의 예측 성향을 분석했습니다.

| 감정 클래스 | 실제 정답(Ground Truth) | Full FT 예측 | LoRA 예측 | 분석 (Insight) |
|:---:|:---:|:---:|:---:|:---|
| **부정 (Negative)** | 88개 | 81개 (-7) | **103개 (+15)** | LoRA는 부정을 다소 과하게 예측함 |
| **중립 (Neutral)** | 99개 | **106개 (+7)** | **70개 (-29)** | **LoRA의 약점**. 애매한 리뷰를 중립으로 보지 못함 |
| **긍정 (Positive)** | 267개 | 267개 (0) | 281개 (+14) | 두 모델 모두 긍정은 잘 맞춤 |

- **Full FT**: 실제 분포와 매우 유사하게 예측하며, 특히 **중립(Neutral)** 클래스를 잘 잡아냄.
- **LoRA**: **'양극화 현상'**이 관찰됨. 애매한(중립) 데이터를 **부정이나 긍정으로 쏠려서 판단**하는 경향이 있음. (중립 → 부정/긍정 오분류 증가)

---

## 4. 정성적 분석 (Case Study)
복합적인 감정이 섞인 문장에 대해 모델이 어떻게 반응하는지 확인했습니다.

### ① 복합 감정 처리 ("배송 늦음(부정) + 옷 예쁨(긍정)")
> *문장: "배송이 늦어서 화날 뻔했는데, 막상 입어보니 예뻐서 용서가 되네요."*
- **Full FT**: **긍정 88%** (확신 높음)
- **LoRA**: **긍정 65%** / 중립 27% (다소 주저함)
- **해석**: 문맥상 '용서가 된다'는 긍정 결론을 FT가 더 명확하게 파악했습니다. LoRA는 배송 지연 키워드 때문에 중립 확률이 섞였습니다.

### ② 성능 불만 처리 ("디자인 예쁨(긍정) + 마감 엉망(부정)")
> *문장: "디자인은 정말 예쁜데, 마감이 너무 엉망이라 실망했어요."*
- **Full FT**: **부정 64%**
- **LoRA**: **부정 70%**
- **해석**: 두 모델 모두 '실망'이라는 부정 키워드에 높은 가중치를 두어 정답(부정)을 잘 맞췄습니다. 의외로 LoRA가 부정 판단은 더 과감했습니다.

### ③ 명확한 문장 처리
- 단순 긍정/부정 문장에서는 두 모델 모두 **85%~99%**의 높은 확신도로 정답을 맞췄습니다.

---

## 5. 결론 및 제언 (Conclusion)

### ✅ 모델 선정 가이드
1. **최고의 성능(Accuracy)이 최우선이라면?** 👉 **Full Fine-Tuning**
   - 특히 **'중립'**이나 **'복합적인 뉘앙스'**를 정확히 구분해야 하는 서비스에 적합합니다.
   
2. **GPU 자원이 부족하거나 빠른 배포가 필요하다면?** 👉 **LoRA**
   - 학습 속도가 2.5배 빠르고 모델 용량이 매우 작습니다.
   - 단, **Epoch를 더 늘리거나(5~10회)**, Rank(r) 값을 조정하여 '중립' 클래스 학습을 보강할 필요가 있습니다.

### 🚀 향후 발전 방향
- **LoRA 튜닝**: 현재 LoRA가 중립을 잘 못 맞추므로, `lora_alpha`나 `r` 값을 높여 표현력을 증대시킬 필요가 있음.
- **데이터 증강**: 중립(Neutral) 데이터가 상대적으로 적으므로, 관련 데이터를 추가 확보하면 성능 향상 기대.

# 회고

1. 안티그래비티 전체로 노트북 코드 작업 진행
  - 전체 코드 작업 후 정성 평가 시각화 정도만 추가로 진행했다
  - 데이터 분석, 통제는 없어지는 느낌
  - 최소 EDA는 직접 or 별도로 진행하는 편이 맞다고 느꼈다.

2. 에이전트 노트북 작업, 배치 디버깅 (로컬(맥북)) - 구글 코랩 (학습) 형태로 진행했을때 경로 관련 오류로 고생했다.
  - 경로 문자열 자소분리 오류 등으로 인식 문제가 많았음
  - 코드 수정 과정에서 분리 되어 있던 코드가 합쳐지면서 지저분해짐
  - 변수명을 새로 지정하는 문제로 이곳저곳 수정도 고생함
  - JSON 파일 내 필드명도 마음대로 다시 수정해서 고침

3. 깔끔한 레포지토리 구조와 재사용성을 고려한 코드작업이 복잡성을 심화시킨다고 느낌
  - 코랩/로컬 환경을 모두 고려해서 상대경로 작업을 지향함
  - 이 과정에서 코드 수정 시 오류나는 경우가 많음
  - 위에서 언급한 한글 폴더명 오류까지 엮여서 더욱 고생함
  - 결과적으로 통제를 잃어버리고 불필요하게 길어진 더티코드...